# Road Centerline Data Analysis Notebook 

In [1]:
# Create Local Processing Environment 
import arcpy
import os
gdb = r"C:\Users\kyleg\Documents\ArcGIS\Projects\CorporateBoundaryAnalysis\CorporateBoundaryAnalysis.gdb"
arcpy.workspace = gdb
arcpy.overwriteOutput = True
roadSourceEF = gdb+r"\RoadCenterlines_17E_F"
roadSourceJ = gdb+r"\RoadCenterlines_17J"
roadTarget = gdb+r"\Roads_GEO_ROADCENTERLINE_NG911_Projected"


In [ ]:
#project the NG911 road centerlines from KDOT SQL server geodatabase to a local processing geodatabase in the Kansas Statwide projection
#The KDOT SQL server geodatabase needs to be updated periodilcally from the KU Oracle geodatabase CHASM
arcpy.management.Project(r"\\gisdata\arcgis\gisdata\Connection_files\Roads\dev\Roads_Dev_geo.sde\Roads.GEO.CHASM_NG911\Roads.GEO.ROADCENTERLINE_NG911", r"C:\gisdata\Roads\Conflate2017.gdb\Roads_GEO_ROADCENTERLINE_NG911_Projected", "PROJCS['NAD_83_Kansas_Lambert_Conformal_Conic_Feet',GEOGCS['GCS_North_American_1983',DATUM['D_North_American_1983',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Lambert_Conformal_Conic'],PARAMETER['false_easting',1312333.333333333],PARAMETER['false_northing',0.0],PARAMETER['central_meridian',-98.25],PARAMETER['standard_parallel_1',37.5],PARAMETER['standard_parallel_2',39.5],PARAMETER['scale_factor',1.0],PARAMETER['latitude_of_origin',36.0],UNIT['Foot_US',0.3048006096012192]]", None, "GEOGCS['GCS_North_American_1983',DATUM['D_North_American_1983',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]", "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")


In [3]:
#put the road centerline data sources for this analysis into the local processing geodatabase
SourceJ = r"C:\gisdata\Roads\Conflate2017.gdb\source_spJ"
SourceEF = r"C:\gisdata\Roads\Conflate2017.gdb\source_spEF"
Target = r"C:\gisdata\Roads\Conflate2017.gdb\target_sp1"

dellist = [SourceJ, SourceEF, Target]
for feature in dellist:
    if arcpy.Exists(feature):
        print("delete "+ feature)
        arcpy.Delete_management(feature)
    else:
        print(feature+" does not exist")


delete C:\gisdata\Roads\Conflate2017.gdb\source_spJ
C:\gisdata\Roads\Conflate2017.gdb\source_spEF does not exist
delete C:\gisdata\Roads\Conflate2017.gdb\target_sp1


In [4]:
#Get some counts of features in the different road centerline feature classes
ef = arcpy.management.GetCount(roadSourceEF)
j = arcpy.management.GetCount(roadSourceJ)
t = arcpy.management.GetCount(roadTarget)
print ("ef:", ef, " j:", j, " t:", t )

ef: 429671  j: 475487  t: 474441


These counts indicate that something is significantly different about version ef.  The J version of conflation data and the target NG911 version are more similar in terms of feature count.

In [5]:
#there probably should not be multipart features, but there are a few
#they are exploded for this analysis.  The multipart features should be reviewed.  

arcpy.overwriteOutput = True
arcpy.management.MultipartToSinglepart(roadTarget, Target)
arcpy.management.MultipartToSinglepart(roadSourceJ, SourceJ)
print ("multipart features exploded")
arcpy.management.EnableCOGO(SourceJ)
arcpy.management.EnableCOGO(Target)
print ("COGO Attributes added")

multipart features exploded
COGO Attributes added


In [6]:
#Get some counts of features in the different road centerline feature classes - exploded
j1 = arcpy.management.GetCount(SourceJ)
t1 = arcpy.management.GetCount(Target)
print (" j:", j1, " t:", t1 )

 j: 475494  t: 474450


In [7]:
#calculate geometry parameters to fields - Shape Length in Feet, Azimuth Angle
#reformat the target Unique Segment ID's so they are the same as in the source
arcpy.management.CalculateField(SourceJ, "NGSEGID", '!GCID!.replace(" ", "")', "PYTHON_9.3", None)
print("segment ID reformatted")
arcpy.management.CalculateField(SourceJ, "Distance", "!Shape_Length!", "PYTHON_9.3", None)
print ("length 1 calculated")
arcpy.management.CalculateField(Target, "Distance", "!Shape_Length!", "PYTHON_9.3", None)
print ("length 2 calculated")
arcpy.management.CalculateField(SourceJ, "Direction", "NorthAzimuth( !SHAPE! )", "PYTHON_9.3", r"def NorthAzimuth(Pline):  \n  degBearing = math.degrees(math.atan2((Pline.lastPoint.X - Pline.firstPoint.X),(Pline.lastPoint.Y - Pline.firstPoint.Y)))  \n  if (degBearing < 0):  \n      degBearing += 360.0  \n  return degBearing  \n")
print("direction aziuth calculated 1")
arcpy.management.CalculateField(Target, "Direction", "NorthAzimuth( !SHAPE! )", "PYTHON_9.3", r"def NorthAzimuth(Pline):  \n  degBearing = math.degrees(math.atan2((Pline.lastPoint.X - Pline.firstPoint.X),(Pline.lastPoint.Y - Pline.firstPoint.Y)))  \n  if (degBearing < 0):  \n      degBearing += 360.0  \n  return degBearing  \n")
print("direction aziuth calculated 2")

segment ID reformatted
length 1 calculated
length 2 calculated
direction aziuth calculated 1
direction aziuth calculated 2


In [8]:
#Once points are created, they can be joined and analyzed
#for future conflations, this cell will delete existing feature classes as needed
#use this if overwrite output doesnt work, which it doesnt seem to do using Jupyter

SourcePT = r"C:\gisdata\Roads\Conflate2017.gdb\source_sp_pt"
TargetPT = r"C:\gisdata\Roads\Conflate2017.gdb\target_sp_pt"

if arcpy.Exists(SourcePT):
    print("delete SourcePT")
    arcpy.Delete_management(SourcePT)
else:
    print(SourcePT+" does not exist")
    
if arcpy.Exists(TargetPT):
    print("delete TargetPT")
    arcpy.Delete_management(TargetPT)
else:
    print(TargetPT+" does not exist")

delete SourcePT
delete TargetPT


In [9]:
#Reduce line geometry to point dimensions for faster spatial analysis
#create the points

arcpy.management.FeatureVerticesToPoints(SourceJ, SourcePT, "MID")
print ("Source Point Created")

arcpy.management.FeatureVerticesToPoints(Target, TargetPT, "MID")
print ("Target Point Created")


Source Point Created
Target Point Created


note that source points arent really being used for anything here.
by definition, the target is the line feature class.

points to points could be analyzed for faster change detection.
none of these methods is 100% reliable for change detection,  assumptions are being made about feature similarity.

there are a few cases where changes might not get detected, like where a line is reshaped but has similar summary properties.

shape vertex matching would take much longer using these methods.

we are assuming that these types of changes are allowable for our purpose of conflating LRS Key information.

Mileage is another story, mileage has to be calibrated after initial Key matches are defined.

The risk of measure error from this conflation is mitigated by scoring match confidence while reducing match criteria to allow more matches that could potentially introduce error.  



In [10]:
#First spatial join - source point to closest target line

pointIdentityL = r"C:\gisdata\Roads\Conflate2017.gdb\KDOT_RoadCenterlines"
if arcpy.Exists(pointIdentityL):
    arcpy.Delete_management(pointIdentityL)

print ("starting spatial join")
arcpy.analysis.SpatialJoin(Target, SourcePT, pointIdentityL, "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "CLOSEST", None, "DistanceToClosest")
print ("adding comparison fields")
arcpy.management.AddField(pointIdentityL, "AzimuthDifference", "DOUBLE", None, None, None, None, "NULLABLE", "NON_REQUIRED", None)
arcpy.management.AddField(pointIdentityL, "DistanceDifference", "DOUBLE", None, None, None, None, "NULLABLE", "NON_REQUIRED", None)
arcpy.management.AddField(pointIdentityL, "MatchScore", "DOUBLE", None, None, None, None, "NULLABLE", "NON_REQUIRED", None)
arcpy.management.AddField(pointIdentityL, "FlipFlag", "Integer", None, None, None, None, "NULLABLE", "NON_REQUIRED", None)
print ("calculating comparison fields")
arcpy.management.CalculateField(pointIdentityL, "DistanceDifference", "abs(!Distance!-!Distance_1!)", "PYTHON_9.3", None)
arcpy.management.CalculateField(pointIdentityL, "AzimuthDifference", "abs(!Direction!-!Direction_1!)", "PYTHON_9.3", None)
#note on the last run, the comparison fields were added but did not calculate correctly
#

starting spatial join
adding comparison fields
calculating comparison fields


<Result 'C:\\gisdata\\Roads\\Conflate2017.gdb\\KDOT_RoadCenterlines'>

In [11]:
#Criteria for setting a 100% match confidence
#same segment ID, location, shape length, azimuth angle
#also look for flipped angles - those are matches, just flag the flip operation

MatchConfidence = '100'
deltaSegmentID = "NGSEGID_1 = NGKSSEGID"
deltaLength ="DistanceDifference < 1" 
deltaDirectionFlip = "AzimuthDifference > 175 AND AzimuthDifference < 185" 
deltaDirection = "(AzimuthDifference > 355 OR AzimuthDifference < 5)"
deltaLocation = "DistanceToClosest < 1"
scoredhigher = "1=1"
criteriaSame = deltaSegmentID+" and "+ deltaLength+" and "+ deltaDirection+" and "+ deltaLocation
criteriaFlipped = deltaSegmentID+" and "+ deltaLength+" and "+ deltaDirectionFlip+" and "+ deltaLocation
print ("criteria set "+criteriaSame)
arcpy.management.MakeFeatureLayer(pointIdentityL, "Confidence100F", criteriaFlipped, None, "#")
arcpy.management.MakeFeatureLayer(pointIdentityL, "Confidence100", criteriaSame, None, "#")
print ("ideal:"+str(arcpy.management.GetCount("Confidence100")), "flipped:"+str(arcpy.management.GetCount("Confidence100F")))

criteria set NGSEGID_1 = NGKSSEGID and DistanceDifference < 1 and (AzimuthDifference > 355 OR AzimuthDifference < 5) and DistanceToClosest < 1
ideal:405315 flipped:706


In [12]:
#calculate 100% confidence scores

arcpy.management.CalculateField("Confidence100", "MatchScore", MatchConfidence, "PYTHON_9.3", None)
arcpy.management.CalculateField("Confidence100F", "FlipFlag", "1", "PYTHON_9.3", None)
arcpy.management.CalculateField("Confidence100F", "MatchScore", MatchConfidence, "PYTHON_9.3", None)


<Result 'Confidence100F'>

In [13]:
#Criteria for setting a 98% match
#same segment ID, 
#similar location, shape length, azimuth angle
#these are mostly lines have had minor geometry edits

MatchConfidence = '98'
deltaSegmentID = "NGSEGID_1 = NGKSSEGID"
deltaLength ="DistanceDifference < 50" 
deltaDirectionFlip = "AzimuthDifference > 170 AND AzimuthDifference < 190" 
deltaDirection = "(AzimuthDifference > 350 OR AzimuthDifference < 10)"
deltaLocation = "DistanceToClosest < 20"
scoredhigher = "(MatchScore < 98 OR MatchScore is null)"
criteriaSame = scoredhigher+" and "+ deltaSegmentID+" and "+ deltaLength+" and "+ deltaDirection+" and "+ deltaLocation
criteriaFlipped = scoredhigher+" and "+deltaSegmentID+" and "+ deltaLength+" and "+ deltaDirectionFlip+" and "+ deltaLocation
print ("criteria set "+criteriaSame)
arcpy.management.MakeFeatureLayer(pointIdentityL, "Confidence98", criteriaSame, None, "#")
arcpy.management.MakeFeatureLayer(pointIdentityL, "Confidence98F", criteriaFlipped, None, "#")
print ("match:"+str(arcpy.management.GetCount("Confidence98")), "flipped:"+str(arcpy.management.GetCount("Confidence98F")))

criteria set (MatchScore < 99 OR MatchScore is null) and NGSEGID_1 = NGKSSEGID and DistanceDifference < 50 and (AzimuthDifference > 350 OR AzimuthDifference < 10) and DistanceToClosest < 20
match:17606 flipped:66


In [14]:
#calculate 98% confidence scores

arcpy.management.CalculateField("Confidence98", "MatchScore", MatchConfidence, "PYTHON_9.3", None)
arcpy.management.CalculateField("Confidence98F", "MatchScore", MatchConfidence, "PYTHON_9.3", None)
arcpy.management.CalculateField("Confidence98F", "FlipFlag", "1", "PYTHON_9.3", None)

<Result 'Confidence98F'>

In [15]:
#Criteria for setting a 96% match
#same segment ID, 
#similar location, shape length, azimuth angle
#these are mostly lines have had significant geometry edits but are still mostly in the same place, direction, and length

MatchConfidence = '96'
deltaSegmentID = "NGSEGID_1 = NGKSSEGID"
deltaLength ="DistanceDifference < 100" 
deltaDirectionFlip = "AzimuthDifference > 170 AND AzimuthDifference < 190" 
deltaDirection = "(AzimuthDifference > 350 OR AzimuthDifference < 10)"
deltaLocation = "DistanceToClosest < 50"
scoredhigher = "(MatchScore < 96 OR MatchScore is null)"
criteriaSame = scoredhigher+" and "+ deltaSegmentID+" and "+ deltaLength+" and "+ deltaDirection+" and "+ deltaLocation
criteriaFlipped = scoredhigher+" and "+deltaSegmentID+" and "+ deltaLength+" and "+ deltaDirectionFlip+" and "+ deltaLocation
print ("criteria set "+criteriaSame)
arcpy.management.MakeFeatureLayer(pointIdentityL, "Confidence96", criteriaSame, None, "#")
arcpy.management.MakeFeatureLayer(pointIdentityL, "Confidence96F", criteriaFlipped, None, "#")
print ("ideal:"+str(arcpy.management.GetCount("Confidence96")), "flipped:"+str(arcpy.management.GetCount("Confidence96F")))

criteria set (MatchScore < 97 OR MatchScore is null) and NGSEGID_1 = NGKSSEGID and DistanceDifference < 100 and (AzimuthDifference > 350 OR AzimuthDifference < 10) and DistanceToClosest < 50
ideal:839 flipped:3


In [16]:
#calculate 96% confidence scores

arcpy.management.CalculateField("Confidence96", "MatchScore", MatchConfidence, "PYTHON_9.3", None)
arcpy.management.CalculateField("Confidence96F", "MatchScore", MatchConfidence, "PYTHON_9.3", None)
arcpy.management.CalculateField("Confidence96F", "FlipFlag", "1", "PYTHON_9.3", None)

<Result 'Confidence96F'>

In [17]:
#Criteria for setting a 95% match confidence
#same location, shape length, azimuth angle with very tight tolerance
#also look for flipped angles - those are matches, just flag the flip operation
#no segment ID relationship - some segment ID's might be null, those will be included

MatchConfidence = '95'
deltaSegmentID = "NGSEGID_1 <> NGKSSEGID"
deltaLength ="DistanceDifference < 1" 
deltaDirectionFlip = "AzimuthDifference > 175 AND AzimuthDifference < 185" 
deltaDirection = "(AzimuthDifference > 355 OR AzimuthDifference < 5)"
deltaLocation = "DistanceToClosest < 1"
scoredhigher = "(MatchScore < 95 OR MatchScore is null)"
criteriaSame = scoredhigher+" and "+ deltaLength+" and "+ deltaDirection+" and "+ deltaLocation 
criteriaFlipped = scoredhigher+" and "+deltaLength+" and "+ deltaDirectionFlip +" and "+ deltaLocation
print ("criteria set "+criteriaSame)
arcpy.management.MakeFeatureLayer(pointIdentityL, "Confidence95", criteriaSame, None, "#")
arcpy.management.MakeFeatureLayer(pointIdentityL, "Confidence95F", criteriaFlipped, None, "#")
print ("match:"+str(arcpy.management.GetCount("Confidence95")), "flipped:"+str(arcpy.management.GetCount("Confidence95F")))

criteria set (MatchScore < 96 OR MatchScore is null) and DistanceDifference < 1 and (AzimuthDifference > 355 OR AzimuthDifference < 5) and DistanceToClosest < 1
match:27784 flipped:496


In [18]:
#calculate 95% confidence scores

arcpy.management.CalculateField("Confidence95", "MatchScore", MatchConfidence, "PYTHON_9.3", None)
arcpy.management.CalculateField("Confidence95F", "MatchScore", MatchConfidence, "PYTHON_9.3", None)
arcpy.management.CalculateField("Confidence95F", "FlipFlag", "1", "PYTHON_9.3", None)

<Result 'Confidence95F'>

In [19]:
#Criteria for setting a 90% match confidence
#same location, shape length, azimuth angle with close tolerance
#also look for flipped angles - those are matches, just flag the flip operation
#no segment ID relationship - some segment ID's might be null, those will be included

MatchConfidence = '90'
deltaSegmentID = "NGSEGID_1 <> NGKSSEGID"
deltaLength ="DistanceDifference < 50" 
deltaDirectionFlip = "AzimuthDifference > 175 AND AzimuthDifference < 185" 
deltaDirection = "(AzimuthDifference > 355 OR AzimuthDifference < 5)"
deltaLocation = "DistanceToClosest < 20"
scoredhigher = "(MatchScore < 90 OR MatchScore is null)"
criteriaSame = scoredhigher+" and "+ deltaLength+" and "+ deltaDirection+" and "+ deltaLocation 
criteriaFlipped = scoredhigher+" and "+deltaLength+" and "+ deltaDirectionFlip +" and "+ deltaLocation
print ("criteria set "+criteriaSame)
arcpy.management.MakeFeatureLayer(pointIdentityL, "Confidence90", criteriaSame, None, "#")
arcpy.management.MakeFeatureLayer(pointIdentityL, "Confidence90F", criteriaFlipped, None, "#")
print ("match:"+str(arcpy.management.GetCount("Confidence90")), "flipped:"+str(arcpy.management.GetCount("Confidence90F")))

criteria set (MatchScore < 95 OR MatchScore is null) and DistanceDifference < 50 and (AzimuthDifference > 355 OR AzimuthDifference < 5) and DistanceToClosest < 20
match:7073 flipped:455


In [20]:
#calculate 90% confidence scores

arcpy.management.CalculateField("Confidence90", "MatchScore", MatchConfidence, "PYTHON_9.3", None)
arcpy.management.CalculateField("Confidence90F", "MatchScore", MatchConfidence, "PYTHON_9.3", None)
arcpy.management.CalculateField("Confidence90F", "FlipFlag", "1", "PYTHON_9.3", None)

<Result 'Confidence90F'>

In [19]:
#Criteria for setting a 88% match confidence
#same location, azimuth angle with close tolerance
#also look for flipped angles - those are matches, just flag the flip operation
#no segment ID relationship - some segment ID's might be null, those will be included
#many of the remaining issues are many to one relationships
#the shape lengths will be very different but the azimuth and the nearest segment will be a close match


MatchConfidence = '88'
deltaSegmentID = "NGSEGID_1 <> NGKSSEGID"
deltaDirectionFlip = "AzimuthDifference > 175 AND AzimuthDifference < 185" 
deltaDirection = "(AzimuthDifference > 355 OR AzimuthDifference < 5)"
deltaLocation = "DistanceToClosest < 2"
scoredhigher = "(MatchScore < 88 OR MatchScore is null)"
criteriaSame = scoredhigher+" and "+ deltaLength+" and "+ deltaDirection+" and "+ deltaLocation 
criteriaFlipped = scoredhigher+" and "+deltaLength+" and "+ deltaDirectionFlip +" and "+ deltaLocation
print ("criteria set "+criteriaSame)
arcpy.management.MakeFeatureLayer(pointIdentityL, "Confidence88", criteriaSame, None, "#")
arcpy.management.MakeFeatureLayer(pointIdentityL, "Confidence88F", criteriaFlipped, None, "#")
print ("match:"+str(arcpy.management.GetCount("Confidence88")), "flipped:"+str(arcpy.management.GetCount("Confidence88F")))

criteria set (MatchScore < 95 OR MatchScore is null) and DistanceDifference < 50 and (AzimuthDifference > 355 OR AzimuthDifference < 5) and DistanceToClosest < 20
match:7073 flipped:455


In [20]:
#calculate 88% confidence scores

arcpy.management.CalculateField("Confidence88", "MatchScore", MatchConfidence, "PYTHON_9.3", None)
arcpy.management.CalculateField("Confidence88F", "MatchScore", MatchConfidence, "PYTHON_9.3", None)
arcpy.management.CalculateField("Confidence88F", "FlipFlag", "1", "PYTHON_9.3", None)

<Result 'Confidence90F'>

In [43]:
  
pointIdentity2 = r"C:\gisdata\Roads\Conflate2017.gdb\SecondJoin"
if arcpy.Exists(pointIdentity2):
    arcpy.Delete_management(pointIdentity2)
    print("deleted") 
else:
    print ("SecondJoin does not exist")
    
pointIdentityM = r"C:\gisdata\Roads\Conflate2017.gdb\KDOT_RoadCenterlines2"
if arcpy.Exists(pointIdentityM):
    arcpy.Delete_management(pointIdentityM)
    print ("Deleted")
else:
    print ("KDOT_RoadCenterlines2 does not exist")
    


SecondJoin does not exist
Deleted
KDOT_RoadCenterlines2I does not exist


In [44]:
#need to eliminate the fieids from the first spatial join, easy way to do that in pro is FC to FC 
scoredhigher = "(MatchScore < 90 OR MatchScore is null)"
#arcpy.management.MakeFeatureLayer(pointIdentityL, "SecondJoin", scoredhigher, None, "#")   
pointIdentityM = r"C:\gisdata\Roads\Conflate2017.gdb\KDOT_RoadCenterlines2"
arcpy.conversion.FeatureClassToFeatureClass("SecondJoin", r"C:\gisdata\Roads\Conflate2017.gdb", "KDOT_RoadCenterlines2", scoredhigher, 'TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,SecondJoin,TARGET_FID,-1,-1;STATE_L "STATE_L" true true false 2 Text 0 0,First,#,SecondJoin,STATE_L,0,2;STATE_R "STATE_R" true true false 2 Text 0 0,First,#,SecondJoin,STATE_R,0,2;COUNTY_L "COUNTY_L" true true false 40 Text 0 0,First,#,SecondJoin,COUNTY_L,0,40;COUNTY_R "COUNTY_R" true true false 40 Text 0 0,First,#,SecondJoin,COUNTY_R,0,40;MUNI_L "MUNI_L" true true false 100 Text 0 0,First,#,SecondJoin,MUNI_L,0,100;MUNI_R "MUNI_R" true true false 100 Text 0 0,First,#,SecondJoin,MUNI_R,0,100;L_F_ADD "L_F_ADD" true true false 4 Long 0 0,First,#,SecondJoin,L_F_ADD,-1,-1;L_T_ADD "L_T_ADD" true true false 4 Long 0 0,First,#,SecondJoin,L_T_ADD,-1,-1;R_F_ADD "R_F_ADD" true true false 4 Long 0 0,First,#,SecondJoin,R_F_ADD,-1,-1;R_T_ADD "R_T_ADD" true true false 4 Long 0 0,First,#,SecondJoin,R_T_ADD,-1,-1;POSTCO_L "POSTCO_L" true true false 40 Text 0 0,First,#,SecondJoin,POSTCO_L,0,40;POSTCO_R "POSTCO_R" true true false 40 Text 0 0,First,#,SecondJoin,POSTCO_R,0,40;ZIP_L "ZIP_L" true true false 5 Text 0 0,First,#,SecondJoin,ZIP_L,0,5;ZIP_R "ZIP_R" true true false 5 Text 0 0,First,#,SecondJoin,ZIP_R,0,5;ESN_L "ESN_L" true true false 5 Text 0 0,First,#,SecondJoin,ESN_L,0,5;ESN_R "ESN_R" true true false 5 Text 0 0,First,#,SecondJoin,ESN_R,0,5;MSAGCO_L "MSAGCO_L" true true false 30 Text 0 0,First,#,SecondJoin,MSAGCO_L,0,30;MSAGCO_R "MSAGCO_R" true true false 30 Text 0 0,First,#,SecondJoin,MSAGCO_R,0,30;STP "STP" true true false 20 Text 0 0,First,#,SecondJoin,STP,0,20;SPDLIMIT "SPDLIMIT" true true false 4 Long 0 0,First,#,SecondJoin,SPDLIMIT,-1,-1;LABEL "LABEL" true true false 121 Text 0 0,First,#,SecondJoin,LABEL,0,121;ELEV_F "ELEV_F" true true false 4 Long 0 0,First,#,SecondJoin,ELEV_F,-1,-1;ELEV_T "ELEV_T" true true false 4 Long 0 0,First,#,SecondJoin,ELEV_T,-1,-1;SURFACE "SURFACE" true true false 10 Text 0 0,First,#,SecondJoin,SURFACE,0,10;STATUS "STATUS" true true false 10 Text 0 0,First,#,SecondJoin,STATUS,0,10;TRAVEL "TRAVEL" true true false 20 Text 0 0,First,#,SecondJoin,TRAVEL,0,20;SHAPE_LENG "SHAPE_LENG" true true false 8 Double 0 0,First,#,SecondJoin,SHAPE_LENG,-1,-1;LRSKEY "LRSKEY" true true false 24 Text 0 0,First,#,SecondJoin,LRSKEY,0,24;EXCEPTION "EXCEPTION" true true false 20 Text 0 0,First,#,SecondJoin,EXCEPTION,0,20;SUBMIT "SUBMIT" true true false 1 Text 0 0,First,#,SecondJoin,SUBMIT,0,1;NOTES "NOTES" true true false 254 Text 0 0,First,#,SecondJoin,NOTES,0,254;UNINC_L "UNINC_L" true true false 100 Text 0 0,First,#,SecondJoin,UNINC_L,0,100;UNINC_R "UNINC_R" true true false 100 Text 0 0,First,#,SecondJoin,UNINC_R,0,100;AUTH_L "AUTH_L" true true false 1 Text 0 0,First,#,SecondJoin,AUTH_L,0,1;AUTH_R "AUTH_R" true true false 1 Text 0 0,First,#,SecondJoin,AUTH_R,0,1;GEOMSAGL "GEOMSAGL" true true false 1 Text 0 0,First,#,SecondJoin,GEOMSAGL,0,1;GEOMSAGR "GEOMSAGR" true true false 1 Text 0 0,First,#,SecondJoin,GEOMSAGR,0,1;RD_RENDER "RD_RENDER" true true false 50 Text 0 0,First,#,SecondJoin,RD_RENDER,0,50;NGKSSEGID "NGKSSEGID" true true false 44 Text 0 0,First,#,SecondJoin,NGKSSEGID,0,44;STP_RD "STP_RD" true true false 80 Text 0 0,First,#,SecondJoin,STP_RD,0,80;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,SecondJoin,ORIG_FID,-1,-1;Direction "Direction" true true false 8 Double 0 0,First,#,SecondJoin,Direction,-1,-1;Distance "Distance" true true false 8 Double 0 0,First,#,SecondJoin,Distance,-1,-1;Radius "Radius" true true false 8 Double 0 0,First,#,SecondJoin,Radius,-1,-1;ArcLength "Arc Length" true true false 8 Double 0 0,First,#,SecondJoin,ArcLength,-1,-1;Radius2 "Radius2" true true false 8 Double 0 0,First,#,SecondJoin,Radius2,-1,-1;SHAPE_Length "SHAPE_Length" false true true 8 Double 0 0,First,#,SecondJoin,SHAPE_Length,-1,-1;AzimuthDifference "AzimuthDifference" true true false 8 Double 0 0,First,#,SecondJoin,AzimuthDifference,-1,-1;DistanceDifference "DistanceDifference" true true false 8 Double 0 0,First,#,SecondJoin,DistanceDifference,-1,-1;MatchScore "MatchScore" true true false 8 Double 0 0,First,#,SecondJoin,MatchScore,-1,-1;FlipFlag "FlipFlag" true true false 4 Long 0 0,First,#,SecondJoin,FlipFlag,-1,-1', None)
#14107 unmatched for the 1:M join

print ("still unmatched:"+str(arcpy.management.GetCount(pointIdentityM)))



still unmatched:14107


In [48]:
#Second spatial join - source point to target line within a mile
#start with matches less %
print ("starting spatial join")
pointIdentityI = r"C:\gisdata\Roads\Conflate2017.gdb\KDOT_RoadCenterlines2I"
if arcpy.Exists(pointIdentityI):
    arcpy.Delete_management(pointIdentityI)
    print ("deleted")
else:
    print ("KDOT_RoadCenterlines2I does not exist")

pointIdentityI = r"C:\gisdata\Roads\Conflate2017.gdb\KDOT_RoadCenterlines2I"
arcpy.analysis.SpatialJoin(pointIdentityM, SourcePT, pointIdentityI, "JOIN_ONE_TO_MANY", "KEEP_ALL", '#', "Within a distance", "1 Miles", "DistanceTo")
#Distance To is not calculating with the  "Within a distance" approach
#record count is about 2.337 million with 14107 features joined to all pts within a mile


#arcpy.management.AddField(pointIdentityM, "AzimuthDifference", "DOUBLE", None, None, None, None, "NULLABLE", "NON_REQUIRED", None)
#arcpy.management.AddField(pointIdentityM, "DistanceDifference", "DOUBLE", None, None, None, None, "NULLABLE", "NON_REQUIRED", None)
#arcpy.management.AddField(pointIdentityM, "MatchScore", "DOUBLE", None, None, None, None, "NULLABLE", "NON_REQUIRED", None)
#arcpy.management.AddField(pointIdentityM, "FlipFlag", "Integer", None, None, None, None, "NULLABLE", "NON_REQUIRED", None)
print ("calculating comparison fields for "+arcpy.management.GetCount(pointIdentityI)+" rows.")
arcpy.management.CalculateField(pointIdentityI, "DistanceDifference", "abs(!Distance!-!Distance_1!)", "PYTHON_9.3", None)
arcpy.management.CalculateField(pointIdentityI, "AzimuthDifference", "abs(!Direction!-!Direction_1!)", "PYTHON_9.3", None)
#note on the last run, the comparison fields were added but did not calculate correctly

starting spatial join
deleted


TypeError: Can't convert 'Result' object to str implicitly

In [ ]:
#Criteria for setting a 85% match confidence
#same nearest within a distance of 1 miles
#matching segment ID relationship and similar geometry
#calculate count of many to 1 matches 

MatchConfidence = '85'
deltaSegmentID = "NGSEGID_1 = NGKSSEGID"
deltaLength ="DistanceDifference < 150" 
deltaDirectionFlip = "(AzimuthDifference > 170 AND AzimuthDifference < 190)" 
deltaDirection = "(AzimuthDifference > 355 OR AzimuthDifference < 5)"
criteriaSame = deltaLength+" and "+ deltaDirection+" and "+ deltaSegmentID 
criteriaFlipped = deltaLength+" and "+ deltaDirectionFlip +" and "+ deltaSegmentID 
print ("criteria set "+criteriaSame)
arcpy.management.MakeFeatureLayer(pointIdentityI, "Confidence85", criteriaSame, None, "#")
arcpy.management.MakeFeatureLayer(pointIdentityI, "Confidence85F", criteriaFlipped, None, "#")
print ("match:"+str(arcpy.management.GetCount("Confidence85")), "flipped:"+str(arcpy.management.GetCount("Confidence85F")))

In [ ]:
#calculate 85% confidence scores

arcpy.management.CalculateField("Confidence85", "MatchScore", MatchConfidence, "PYTHON_9.3", None)
arcpy.management.CalculateField("Confidence85F", "MatchScore", MatchConfidence, "PYTHON_9.3", None)
arcpy.management.CalculateField("Confidence85F", "FlipFlag", "1", "PYTHON_9.3", None)